In [39]:
import pandas as pd
import plotly.offline as offline
import plotly.graph_objs as go
import plotly_express as px

# Emissions Stacked Bar Chart

In [40]:
#load data
BEIS_csv = pd.read_excel('Data/LA_Emissions.xlsx',engine = 'openpyxl')
#clean na rows
BEIS_csv = BEIS_csv[BEIS_csv['Calendar Year'].notna()]
# Select Domestic emissions
Housing_Total = BEIS_csv[['Calendar Year','Domestic Electricity','Domestic Gas',"Domestic 'Other'"]]
Housing_Total.rename(columns={'Domestic Electricity': 'Electricity',
                              'Domestic Gas': 'Gas',
                              "Domestic 'Other'": 'Other'
                             }, inplace=True)
#melt into longform
Housing_Total = pd.melt(Housing_Total, id_vars='Calendar Year', value_vars=[
                                                                  'Electricity',
                                                                  'Gas' ,
                                                                  'Other',
                                                                  ])
# convert datetime to year
Housing_Total['Year'] = pd.to_datetime(Housing_Total['Calendar Year']).dt.year
#rename columns
Housing_Total.rename(columns={'variable': 'Housing Emissions',
                              'value': 'GHG Emissions (TCO2)',
                             }, inplace=True)

C:\Users\ACAL0920\Documents\Anaconda\envs\myenv\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
color_range = ["#404bb7", "#87c7ec",'#76bd22']

housing_fig = px.bar(Housing_Total, x="Year", y="GHG Emissions (TCO2)", color="Housing Emissions",
                     color_discrete_sequence=color_range,
                    height = 1000
                    )

housing_fig.update_layout(
    font={"family": "Poppins",'color': '#000f9f'},
    font_size=12,
    margin=dict(
        l=0,   # left margin
        r=0,   # right margin
        t=5,   # top margin
        b=810    # bottom margin
    ),
    legend=dict(
        orientation="v",  # Set the orientation to horizontal
        yanchor="auto",    # Anchor the legend to the top
        y=1.1 # Position the legend slightly above the top of the graph
    ),
)


housing_fig.update_xaxes(title=None)
housing_fig.update_yaxes(title=None)

housing_fig.show()

In [42]:
housing_fig.write_html('Blocks/Housing_Emissions_Bar.html',include_plotlyjs=False)

# Key Indicator plot

In [43]:
#load data
BEIS_csv = pd.read_excel('Data/LA_Emissions.xlsx', engine='openpyxl')
Grid_csv  = pd.read_excel('Data/LA_Emissions.xlsx', engine='openpyxl', sheet_name='Grid_factors')
#clean na rows
BEIS_csv = BEIS_csv[BEIS_csv['Calendar Year'].notna()]
# Select Domestic emissions
Domestic_Total = BEIS_csv[['Calendar Year','Grand Total','Domestic Total','Domestic Gas','Domestic Electricity']]
Domestic_Total['Calendar Year'] = pd.to_datetime(Domestic_Total['Calendar Year']).dt.year

C:\Users\ACAL0920\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [44]:
# Indicator 2: Domestic Emissions
years = Domestic_Total['Calendar Year'].unique() #create a list of years

second_to_last_year = sorted(years)[-2] #sort to identify last & second to last
latest_year = sorted(years)[-1]

domestic_previous_value = Domestic_Total.loc[Domestic_Total['Calendar Year'] == second_to_last_year, 'Domestic Total'].values[0]
domestic_current_value = Domestic_Total.loc[Domestic_Total['Calendar Year'] == latest_year, 'Domestic Total'].values[0]

In [45]:
# Indicator 3: EPC Band C 
indicators = pd.read_excel('Data/indicators.xlsx', engine='openpyxl', sheet_name='LA_Indicators')

indicators = indicators[indicators['Measure'] == "Percentage of dwellings with EPC Band 'C' or above "]

years = indicators['Year'].unique() #create a list of years
second_to_last_year = sorted(years)[-2] #sort to identify last & second to last
latest_year = sorted(years)[-1]

#use year list to identify values
EPC_previous_value = indicators.loc[indicators['Year'] == second_to_last_year, 'Value'].values[0]
EPC_current_value = indicators.loc[indicators['Year'] == latest_year, 'Value'].values[0]

In [46]:
# Indicator 4: Efficiency Installs
installs_csv = pd.read_csv('Data/LA_efficiency_measures.csv')

years = installs_csv['Financial Year End'].unique() #create a list of years

second_to_last_year = sorted(years)[-2] #sort to identify last & second to last
latest_year = sorted(years)[-1]

second_to_last_year = sorted(years)[-2] #sort to identify last & second to last
latest_year = sorted(years)[-1]

#use year list to identify values
Installs_previous_value = installs_csv.loc[installs_csv['Financial Year End'] == second_to_last_year, 'Total Installation of Energy Efficiency Measures'].values[0]
Installs_current_value = installs_csv.loc[installs_csv['Financial Year End'] == latest_year, 'Total Installation of Energy Efficiency Measures'].values[0]

In [47]:
indicator_fig = go.Figure()

# Indicator 2: DOMESTIC
indicator_fig.add_trace(go.Indicator(
    mode="number+delta",
    value=domestic_current_value,
    domain={'x': [0.25, 0.75], 'y': [0.75, 0.95]},
    delta={'reference': domestic_previous_value, 'relative': True,'valueformat': '.1%'},
    title={'text': "Housing CO2 (Tonnes)", 'font': {'family': "Poppins", 'size': 16, 'color': '#000f9f'}},
    number={'font': {'color': '#000f9f'}}
))

# Indicator 3:  Installs
indicator_fig.add_trace(go.Indicator(
    mode="number+delta",
    value= Installs_current_value,
    domain={'x': [0.25, 0.75], 'y': [0.5, 0.7]},
    delta={'reference': Installs_previous_value, 'relative': True,'valueformat': '.1%'},
    title={'text': "NTC Efficiency Measures Installed", 'font': {'family': "Poppins", 'size': 16, 'color': '#000f9f'}},
    number={'font': {'color': '#000f9f'}}
))

# Indicator 4: EPC
indicator_fig.add_trace(go.Indicator(
    mode = "gauge+number+delta",
    value= EPC_current_value,
    domain={'x': [0.1, 0.9], 'y': [0.02, 0.37]},
    gauge = {'axis': {'range': [None, 100]},'bar': {'color': "#76bd22"}},
    delta={'reference':  EPC_previous_value, 'relative': True,'valueformat': '.1%'},
    title={'text': "% Homes above EPC 'C'", 'font': {'family': "Poppins", 'size': 16, 'color': '#000f9f'}},
    number={'font': {'color': '#000f9f'}}
))


indicator_fig.update_layout(
    font={'family': "Poppins"},
    margin=dict(t=15, b=0, l=0,r=0)
    
)


indicator_fig.show()


In [48]:
indicator_fig.write_html('Blocks/Housing_Indictors_Plot.html',include_plotlyjs=False)

# Emissions Tree Chart

In [49]:
#read data
BEIS_csv = pd.read_excel('Data/LA_Emissions.xlsx', engine='openpyxl')
#select totals columns
BEIS_Totals = BEIS_csv[['Calendar Year',
                        'Industry Total',
                        'Commercial Total',
                        'Public Sector Total',
                        'Domestic Total',
                        'Transport Total',
                        'Agriculture Total',
                        'Waste Management Total']]

#select latest year
latest_year = BEIS_Totals.sort_values("Calendar Year", ascending=False).iloc[0]["Calendar Year"]
BEIS_Totals_latest = BEIS_Totals[BEIS_Totals["Calendar Year"] == latest_year]

# Rename Columns
BEIS_Totals_latest.rename(columns={'Industry Total': 'Industry Emissions',
                              'Commercial Total': 'Commercial Emissions',
                              'Public Sector Total': 'Public Sector Emissions',
                              'Domestic Total': 'Housing Emissions',
                              'Transport Total': 'Transport Emissions',
                              'Agriculture Total': 'Agriculture Emissions',
                              'Commercial Total': 'Commercial Emissions',
                              'Waste Management Total': 'Waste Management Emissions',
                             }, inplace=True)

#change wide to long
df_long = pd.melt(BEIS_Totals_latest, id_vars=["Calendar Year"])

#specify colours
color_range = ["#c3d600", "#d7ecf8",'#00c4b4',"#40d3c7",'#5fb4e5','#ffd200']

# Create a treemap chart
emissions_treemap = px.treemap(df_long,
                               values='value',
                               path=['variable'],
                               color='variable',
                               color_discrete_sequence=color_range,
                               height = 1000
                               )
emissions_treemap.update_layout(
    font={'family': "Poppins",
          'color':'#000f9f',
         'size':14},
    margin=dict(t=0, b=810, l=0,r=0),

)

C:\Users\ACAL0920\Documents\Anaconda\envs\myenv\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [50]:
emissions_treemap.write_html('Blocks/Housing_Emissions_Tree.html',include_plotlyjs=False)

# Installs area plot

In [51]:
#read data
installs_csv = pd.read_csv('Data/LA_efficiency_measures.csv')
# Delete non-relavant column 
installs_data = installs_csv.drop('Total Installation of Energy Efficiency Measures', axis=1)
# change wide to long
installs_data = pd.melt(installs_data, id_vars='Financial Year End')
#rename columns
installs_data.rename(columns={'variable': 'Efficiency Measures Installed',
                              'value': 'Installs',
                             }, inplace=True)
#specify plot colours
color_range = ["#c50f3c", "#76bd22",'#00c4b4','#5fb4e5','#d87900','#ffd200']
#plot areamap
installs_area = px.area(installs_data, x="Financial Year End", y="Installs", color="Efficiency Measures Installed",line_group = "Efficiency Measures Installed",
                     color_discrete_sequence=color_range,height = 1000)

installs_area.update_layout(
    font={"family": "Poppins",'color': '#000f9f'},
    font_size=12,
    margin=dict(
        l=0,   # left margin
        r=0,   # right margin
        t=5,   # top margin
        b=810    # bottom margin
    ),
)


installs_area.update_xaxes(title=None)
installs_area.update_yaxes(title=None)

installs_area.show()

In [52]:
installs_area.write_html('Blocks/Housing_Installs_AreaPlot.html',include_plotlyjs=False)